# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

W0827 01:05:36.070000 2523725 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 01:05:36.070000 2523725 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-27 01:05:37] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=34331, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=78424

[2025-08-27 01:05:38] Using default HuggingFace chat template with detected content format: string


W0827 01:05:46.491000 2524009 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 01:05:46.491000 2524009 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0827 01:05:46.796000 2524010 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 01:05:46.796000 2524010 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

[2025-08-27 01:05:55] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=62.64 GB


[2025-08-27 01:05:55] INFO:     Started server process [2523725]
[2025-08-27 01:05:55] INFO:     Waiting for application startup.
[2025-08-27 01:05:55] INFO:     Application startup complete.
[2025-08-27 01:05:55] INFO:     Uvicorn running on http://0.0.0.0:34331 (Press CTRL+C to quit)


[2025-08-27 01:05:56] INFO:     127.0.0.1:49798 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-27 01:05:56] INFO:     127.0.0.1:49810 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-27 01:05:56] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:05:57] INFO:     127.0.0.1:49822 - "POST /generate HTTP/1.1" 200 OK
[2025-08-27 01:05:57] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-27 01:06:01] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:06:02] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.31, #queue-req: 0, 


[2025-08-27 01:06:02] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.83, #queue-req: 0, 


[2025-08-27 01:06:03] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.59, #queue-req: 0, 


[2025-08-27 01:06:03] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.16, #queue-req: 0, 


...

[2025-08-27 01:06:04] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.91, #queue-req: 0, 


[2025-08-27 01:06:04] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.88, #queue-req: 0, 


[2025-08-27 01:06:04] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, cuda graph: True, gen throughput (token/s): 161.54, #queue-req: 0, 


[2025-08-27 01:06:04] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.87, #queue-req: 0, 


[2025-08-27 01:06:05] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: True, gen throughput (token/s): 159.58, #queue-req: 0, 
[2025-08-27 01:06:05] INFO:     127.0.0.1:49836 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-27 01:06:05] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:06:05] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, cuda graph: True, gen throughput (token/s): 147.91, #queue-req: 0, 


[2025-08-27 01:06:05] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.80, #queue-req: 0, 


[2025-08-27 01:06:05] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.52, #queue-req: 0, 


[2025-08-27 01:06:06] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.17, #queue-req: 0, 


[2025-08-27 01:06:06] Decode batch. #running-req: 1, #token: 216, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.10, #queue-req: 0, 


[2025-08-27 01:06:06] Decode batch. #running-req: 1, #token: 256, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.10, #queue-req: 0, 


[2025-08-27 01:06:06] Decode batch. #running-req: 1, #token: 296, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.88, #queue-req: 0, 


[2025-08-27 01:06:07] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.89, #queue-req: 0, 


[2025-08-27 01:06:07] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.81, #queue-req: 0, 
[2025-08-27 01:06:07] INFO:     127.0.0.1:49836 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-27 01:06:07] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:06:07] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, cuda graph: True, gen throughput (token/s): 146.38, #queue-req: 0, 


[2025-08-27 01:06:07] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.84, #queue-req: 0, 


[2025-08-27 01:06:08] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.29, #queue-req: 0, 


[2025-08-27 01:06:08] Decode batch. #running-req: 1, #token: 181, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.06, #queue-req: 0, 


...

[2025-08-27 01:06:10] Decode batch. #running-req: 1, #token: 581, token usage: 0.03, cuda graph: True, gen throughput (token/s): 163.69, #queue-req: 0, 


[2025-08-27 01:06:11] Decode batch. #running-req: 1, #token: 621, token usage: 0.03, cuda graph: True, gen throughput (token/s): 163.76, #queue-req: 0, 


[2025-08-27 01:06:11] Decode batch. #running-req: 1, #token: 661, token usage: 0.03, cuda graph: True, gen throughput (token/s): 163.70, #queue-req: 0, 


[2025-08-27 01:06:11] Decode batch. #running-req: 1, #token: 701, token usage: 0.03, cuda graph: True, gen throughput (token/s): 163.66, #queue-req: 0, 


[2025-08-27 01:06:11] Decode batch. #running-req: 1, #token: 741, token usage: 0.04, cuda graph: True, gen throughput (token/s): 163.58, #queue-req: 0, 
[2025-08-27 01:06:11] INFO:     127.0.0.1:49836 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-27 01:06:11] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:06:12] Decode batch. #running-req: 1, #token: 31, token usage: 0.00, cuda graph: True, gen throughput (token/s): 147.50, #queue-req: 0, 


[2025-08-27 01:06:12] Decode batch. #running-req: 1, #token: 71, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.89, #queue-req: 0, 


[2025-08-27 01:06:12] Decode batch. #running-req: 1, #token: 111, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.92, #queue-req: 0, 
[2025-08-27 01:06:12] INFO:     127.0.0.1:49836 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-27 01:06:12] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 01:06:12] Decode batch. #running-req: 1, #token: 491, token usage: 0.02, cuda graph: True, gen throughput (token/s): 104.71, #queue-req: 0, 


[2025-08-27 01:06:13] Decode batch. #running-req: 1, #token: 531, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.33, #queue-req: 0, 


[2025-08-27 01:06:13] Decode batch. #running-req: 1, #token: 571, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.01, #queue-req: 0, 


[2025-08-27 01:06:13] Decode batch. #running-req: 1, #token: 611, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.51, #queue-req: 0, 


[2025-08-27 01:06:13] Decode batch. #running-req: 1, #token: 651, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.49, #queue-req: 0, 


...

[2025-08-27 01:06:16] Decode batch. #running-req: 1, #token: 1091, token usage: 0.05, cuda graph: True, gen throughput (token/s): 161.25, #queue-req: 0, 


[2025-08-27 01:06:16] Decode batch. #running-req: 1, #token: 1131, token usage: 0.06, cuda graph: True, gen throughput (token/s): 161.13, #queue-req: 0, 


[2025-08-27 01:06:17] Decode batch. #running-req: 1, #token: 1171, token usage: 0.06, cuda graph: True, gen throughput (token/s): 161.10, #queue-req: 0, 


[2025-08-27 01:06:17] Decode batch. #running-req: 1, #token: 1211, token usage: 0.06, cuda graph: True, gen throughput (token/s): 161.28, #queue-req: 0, 


[2025-08-27 01:06:17] INFO:     127.0.0.1:49836 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-08-27 01:06:18] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 01:06:18] Decode batch. #running-req: 1, #token: 27, token usage: 0.00, cuda graph: True, gen throughput (token/s): 43.41, #queue-req: 0, 


[2025-08-27 01:06:18] Decode batch. #running-req: 1, #token: 67, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.57, #queue-req: 0, 


[2025-08-27 01:06:18] Decode batch. #running-req: 1, #token: 107, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.87, #queue-req: 0, 


[2025-08-27 01:06:19] Decode batch. #running-req: 1, #token: 147, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.08, #queue-req: 0, 


[2025-08-27 01:06:19] Decode batch. #running-req: 1, #token: 187, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.15, #queue-req: 0, 


...

[2025-08-27 01:06:19] Decode batch. #running-req: 1, #token: 267, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.06, #queue-req: 0, 


[2025-08-27 01:06:20] Decode batch. #running-req: 1, #token: 307, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.89, #queue-req: 0, 


[2025-08-27 01:06:20] Decode batch. #running-req: 1, #token: 347, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.84, #queue-req: 0, 


[2025-08-27 01:06:20] Decode batch. #running-req: 1, #token: 387, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.60, #queue-req: 0, 


[2025-08-27 01:06:20] Decode batch. #running-req: 1, #token: 427, token usage: 0.02, cuda graph: True, gen throughput (token/s): 158.92, #queue-req: 0, 
[2025-08-27 01:06:20] INFO:     127.0.0.1:37184 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to identify what the capital of France is. I know that Paris is the capital, so that\'s straightforward. \n\nNext, I need to find the population. I remember that Paris is a major city, so its population is quite large. I think it\'s over 3 million, but I\'m not exactly sure of the exact number. Maybe I should double-check that. \n\nWait, I recall that the population figure can vary depending on the source and the year. The user didn\'t specify a particular year, so I should probably go with the most recent estimate. I believe the population is around 3,500,000 as of 2023. \n\nNow, I need to structure this

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-08-27 01:06:20] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:06:21] Decode batch. #running-req: 1, #token: 62, token usage: 0.00, cuda graph: True, gen throughput (token/s): 155.46, #queue-req: 0, 


[2025-08-27 01:06:21] Decode batch. #running-req: 1, #token: 102, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.87, #queue-req: 0, 


[2025-08-27 01:06:21] Decode batch. #running-req: 1, #token: 142, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.30, #queue-req: 0, 


[2025-08-27 01:06:21] Decode batch. #running-req: 1, #token: 182, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.44, #queue-req: 0, 


[2025-08-27 01:06:22] Decode batch. #running-req: 1, #token: 222, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.19, #queue-req: 0, 


[2025-08-27 01:06:22] Decode batch. #running-req: 1, #token: 262, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.83, #queue-req: 0, 


[2025-08-27 01:06:22] Decode batch. #running-req: 1, #token: 302, token usage: 0.01, cuda graph: True, gen throughput (token/s): 159.17, #queue-req: 0, 


[2025-08-27 01:06:22] Decode batch. #running-req: 1, #token: 342, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.30, #queue-req: 0, 


[2025-08-27 01:06:23] Decode batch. #running-req: 1, #token: 382, token usage: 0.02, cuda graph: True, gen throughput (token/s): 166.55, #queue-req: 0, 
[2025-08-27 01:06:23] INFO:     127.0.0.1:37198 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-08-27 01:06:23] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 01:06:23] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 20, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-27 01:06:23] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 2, #queue-req: 0, 


[2025-08-27 01:06:23] Decode batch. #running-req: 3, #token: 50, token usage: 0.00, cuda graph: True, gen throughput (token/s): 177.00, #queue-req: 0, 


[2025-08-27 01:06:23] Decode batch. #running-req: 3, #token: 170, token usage: 0.01, cuda graph: True, gen throughput (token/s): 476.16, #queue-req: 0, 


[2025-08-27 01:06:23] Decode batch. #running-req: 3, #token: 290, token usage: 0.01, cuda graph: True, gen throughput (token/s): 495.11, #queue-req: 0, 


[2025-08-27 01:06:24] Decode batch. #running-req: 3, #token: 410, token usage: 0.02, cuda graph: True, gen throughput (token/s): 492.43, #queue-req: 0, 


[2025-08-27 01:06:24] Decode batch. #running-req: 3, #token: 530, token usage: 0.03, cuda graph: True, gen throughput (token/s): 489.58, #queue-req: 0, 


[2025-08-27 01:06:24] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, cuda graph: True, gen throughput (token/s): 293.06, #queue-req: 0, 


[2025-08-27 01:06:24] Decode batch. #running-req: 1, #token: 264, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.68, #queue-req: 0, 
[2025-08-27 01:06:24] INFO:     127.0.0.1:37204 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThe user has provided the correct information that the capital of France is Paris. However, the user's request was to provide the information of the capital of France, which is a bit ambiguous. It could be interpreted as wanting more detailed information about Paris, such as its history, culture, landmarks, or government. Alternatively, it might mean they want to confirm that Paris is indeed the capital, which they already know. \n\nTo better assist the user, it would be helpful to ask for clarification. If they want more details about Paris, I can provide a comprehensive overview. If they just need confirmation, then the current response suffices. \n\nAdditionally, I can offer to provide more information o

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-08-27 01:06:24] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:06:25] Decode batch. #running-req: 1, #token: 45, token usage: 0.00, cuda graph: True, gen throughput (token/s): 139.85, #queue-req: 0, 


[2025-08-27 01:06:25] Decode batch. #running-req: 1, #token: 85, token usage: 0.00, cuda graph: True, gen throughput (token/s): 169.48, #queue-req: 0, 


[2025-08-27 01:06:25] Decode batch. #running-req: 1, #token: 125, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.99, #queue-req: 0, 


[2025-08-27 01:06:25] Decode batch. #running-req: 1, #token: 165, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.34, #queue-req: 0, 


...

[2025-08-27 01:06:36] Decode batch. #running-req: 1, #token: 1885, token usage: 0.09, cuda graph: True, gen throughput (token/s): 163.04, #queue-req: 0, 


[2025-08-27 01:06:36] Decode batch. #running-req: 1, #token: 1925, token usage: 0.09, cuda graph: True, gen throughput (token/s): 163.06, #queue-req: 0, 


[2025-08-27 01:06:37] Decode batch. #running-req: 1, #token: 1965, token usage: 0.10, cuda graph: True, gen throughput (token/s): 163.15, #queue-req: 0, 


[2025-08-27 01:06:37] Decode batch. #running-req: 1, #token: 2005, token usage: 0.10, cuda graph: True, gen throughput (token/s): 163.15, #queue-req: 0, 


[2025-08-27 01:06:37] Decode batch. #running-req: 1, #token: 2045, token usage: 0.10, cuda graph: True, gen throughput (token/s): 163.08, #queue-req: 0, 
[2025-08-27 01:06:37] INFO:     127.0.0.1:37210 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-08-27 01:06:37] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-27 01:06:37] Decode batch. #running-req: 1, #token: 54, token usage: 0.00, cuda graph: True, gen throughput (token/s): 151.30, #queue-req: 0, 


[2025-08-27 01:06:38] Decode batch. #running-req: 1, #token: 94, token usage: 0.00, cuda graph: True, gen throughput (token/s): 164.57, #queue-req: 0, 


[2025-08-27 01:06:38] Decode batch. #running-req: 1, #token: 134, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.79, #queue-req: 0, 


[2025-08-27 01:06:38] Decode batch. #running-req: 1, #token: 174, token usage: 0.01, cuda graph: True, gen throughput (token/s): 162.29, #queue-req: 0, 


[2025-08-27 01:06:38] Decode batch. #running-req: 1, #token: 214, token usage: 0.01, cuda graph: True, gen throughput (token/s): 161.97, #queue-req: 0, 


[2025-08-27 01:06:39] Decode batch. #running-req: 1, #token: 254, token usage: 0.01, cuda graph: True, gen throughput (token/s): 161.97, #queue-req: 0, 


[2025-08-27 01:06:39] Decode batch. #running-req: 1, #token: 294, token usage: 0.01, cuda graph: True, gen throughput (token/s): 161.66, #queue-req: 0, 


[2025-08-27 01:06:39] Decode batch. #running-req: 1, #token: 334, token usage: 0.02, cuda graph: True, gen throughput (token/s): 161.66, #queue-req: 0, 
[2025-08-27 01:06:39] INFO:     127.0.0.1:38238 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

W0827 01:06:41.607000 2523445 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 01:06:41.607000 2523445 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0827 01:06:50.204000 2526330 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0827 01:06:50.204000 2526330 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.39s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.29s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.31s/it]



Capturing batches (bs=4 avail_mem=58.61 GB):   0%|          | 0/3 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=58.54 GB): 100%|██████████| 3/3 [00:00<00:00, 10.43it/s]


### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris. Here's some information about Paris:

- It is located in the northern part of France.
- Paris is known as the "City of Light" because of its numerous bridges.
- The Eiffel Tower is one of its famous landmarks.
- Paris is a major center for culture, business, and education.
- It has a rich history dating back to ancient times.
- Paris is divided into 20 arrondissements.

Wait, is this correct? I think I might have made a mistake. Maybe I should double-check some of these points.

Let me think again: Paris is indeed
Prompt: Give me the information of the capital of Germany.
Generated text: 
Sure, here's the information about the capital of Germany:

**Capital of Germany: Berlin**

Berlin is the largest city in Germany by both area and population. It serves as the country's political, cultural, and economic center. The city is renowned for its rich history, vibrant cultural scene, an

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its economic and political significance, as well as cultural and tourism aspects.

99 words.

Okay, I need to write a 99-word information about London as a major global city, covering its economic and political significance, as well as cultural and tourism aspects. Hmm, let me start by brainstorming the key points.

First, London's economic significance. It's the financial capital of the UK and the EU, right? So, it's a global hub for finance, banking, and international trade. Maybe mention how it's a major economic powerhouse with a diverse economy.

Then, political significance. London is the political center of the
Prompt: Please provide information about Paris as a major global city:
Generated text:  location, area, population, main industries, famous landmarks, and architecture. We need to find out about Paris in 10 years, 5 years, and 2 years from now. For each future period, we need to note 

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Alright, so I need to figure out the population of the capital of France. First, I should identify what the capital of France is. From what I remember, Paris is the capital. That makes sense because I've heard people talk about Paris so much in movies and TV shows. 

Now, I need to find out the population of Paris. I think it's a big city, so the population is probably in the millions. I recall that major cities in France like Paris, Lyon, and Marseille have populations over 3 million. I'm not sure about the exact number, though. Maybe I can think of any recent news or articles I've read about population statistics.

Wait, I think I've heard that Paris has a population around 2 million. But I'm not certain. Let me try to break it down. France's total population is about 40 million, right? So if Paris is the

In [19]:
llm.shutdown()